In [1]:
from __future__ import print_function
import sdm as sdmlib
from collections import defaultdict
import random
import string
import time
from math import ceil
from IPython.display import clear_output
#%load_ext line_profiler

empty = ' '
flip_table = string.maketrans('OX', 'XO')

In [2]:
class Player(object):
    def __init__(self, name):
        self.name = name
        self.stats = defaultdict(int)
    
    def on_invalid_move(self):
        raise Exception('Ops')
    
    def on_finish(self, winner, seq):
        pass
    
    def next_move(self, step, board):
        v = []
        for idx, x in enumerate(board):
            if x == empty:
                v.append(idx)
        return random.choice(v)

In [3]:
class HumanPlayer(object):
    def __init__(self, name):
        self.name = name
        self.stats = defaultdict(int)
    
    def on_invalid_move(self):
        raise Exception('Ops')
    
    def on_finish(self, winner, seq):
        pass
    
    def next_move(self, step, board):
        clear_output(wait=True)
        while True:
            print_board(board)
            text = raw_input('Entre com a linha/coluna: ')
            if len(text) != 2:
                continue
            si = text[0]
            sj = text[1]
            i = int(si)-1
            j = int(sj)-1
            idx = 3*i+j
            if board[idx] == empty:
                break
        return idx


In [4]:
class SDMPlayer(object):
    def __init__(self, name, sdm0, sdm1, bs_to_boards, boards_to_bs):
        self.name = name

        self.bs_to_boards = bs_to_boards
        self.boards_to_bs = boards_to_bs
        
        self.sdm0 = sdm0
        self.sdm1 = sdm1
        
        self.on_finish = self.on_finish1
        
        self.reset_stats()
    
    def reset_stats(self):
        self.stats = defaultdict(int)
    
    def on_invalid_move(self):
        raise Exception('Ops')
    
    def board_to_bitstring(self, board):
        bs = self.boards_to_bs.get(board, None)
        if bs is None:
            bs = sdmlib.Bitstring.init_random(self.sdm0.bits)
            self.bs_to_boards[bs] = board
            self.boards_to_bs[board] = bs
            self.sdm0.write(bs, bs)
        return bs
    
    def bitstring_to_board(self, bs):
        board = self.bs_to_boards.get(bs, None)
        return board
    
    def flip(self, board):
        return board.translate(flip_table)
    
    def penalize(self, board, ignore):
        debug = False
        if debug:
            print('board')
            print_board(board)
            print('ignore')
            print_board(ignore)
        for idx, x in enumerate(board):
            if x == empty:
                board2 = board[:idx] + 'X' + board[idx+1:]
                if board2 != ignore:
                    if debug:
                        print('aceito')
                        print_board(board2)
                    bs1 = self.board_to_bitstring(board)
                    bs2 = self.board_to_bitstring(board2)
                    self.sdm1.write(bs1, bs2, weight=2)
                else:
                    if debug:
                        print('Ignorou!')
                        print_board(board2)
        
    
    def on_finish1(self, winner, seq):
        debug = False
        prev = None
        if debug:
            print('Learning...')
            
        if winner is not None:
            if winner == 'X':
                self.penalize(self.flip(seq[-3]), self.flip(seq[-2]))
            else:
                self.penalize(seq[-3], seq[-2])
            
        for step, board in enumerate(seq):
            if winner == 'O':
                board = self.flip(board)

            if prev is not None:
                if debug:
                    print_board(prev)
                    print('')
                    print_board(board)
                    print('--')
                bs1 = self.board_to_bitstring(prev)
                bs2 = self.board_to_bitstring(board)
                if winner is None:
                    self.sdm1.write(bs1, bs2)
                else:
                    weight = int(3.0*step/(len(seq)-1))+1
                    self.sdm1.write(bs1, bs2, weight=weight)
            prev = board
                
    def next_move(self, step, board):
        x = self.sdm_move(step, board)
        if x is not None:
            return x
        return self.random_move(step, board)
    
    def sdm_move(self, step, board, debug=False):
        if self.name == 'O':
            board = self.flip(board)

        bs1 = self.board_to_bitstring(board)
        bs2 = self.sdm1.read(bs1)
        bs3 = self.sdm0.iter_read(bs2, max_iter=20)
        board2 = self.bitstring_to_board(bs3)
        if board2 is None:
            return None
        
        if debug:
            print('Reading...')
            print_board(board)
            print('--')
            print_board(board2)
            print('')
        
        diff = []
        for idx in range(9):
            if board[idx] != board2[idx]:
                if board[idx] == empty and board2[idx] == 'X':
                    diff.append(idx)
        if len(diff) == 1:
            self.stats['sdm'] += 1
            return diff[0]
        return None
        #if diff:
        #    self.stats['sdm'] += 1
        #    return random.choice(diff)
        #return None
    
    def random_move(self, step, board):
        self.stats['random'] += 1
        v = []
        for idx, x in enumerate(board):
            if x == empty:
                v.append(idx)
        return random.choice(v)

In [5]:
def print_board(board):
    i = 0
    while i < len(board):
        print('|' + board[i:i+3] + '|')
        i += 3
    print('')

def check_all_equal(*args):
    if len(set(args)) == 1 and args[0] != empty:
        return True
    return False
        
def check_for_winner(board):
    for i in range(3):
        if check_all_equal(board[3*i+0], board[3*i+1], board[3*i+2]):
            return board[3*i+0]
        
        if check_all_equal(board[3*0+i], board[3*1+i], board[3*2+i]):
            return board[3*0+i]

    if check_all_equal(board[3*0+0], board[3*1+1], board[3*2+2]):
        return board[3*0+0]
    
    if check_all_equal(board[3*0+2], board[3*1+1], board[3*2+0]):
        return board[3*0+2]
    
    return None

def play(p1, p2, shuffle=True):
    board = ' '*9
    end = False
    players = [p1, p2]
    if shuffle:
        random.shuffle(players)
    index = 0
    step = 0
    sequence = [board]
    winner = None
    while winner is None and step < 9:
        cur_player = players[index]
        
        idx = cur_player.next_move(step, board)
        
        if board[idx] != empty:
            cur_player.on_invalid_move()
        
        board = board[:idx] + cur_player.name + board[idx+1:]
        sequence.append(board)
        
        winner = check_for_winner(board)
        
        index = (index+1)%2
        step += 1
    return winner, sequence

In [6]:
#bits, radius = 1000, 451
bits, radius = 256, 103

sample = 1000000
scanner_type = sdmlib.SDM_SCANNER_THREAD

address_space = sdmlib.AddressSpace.init_random(bits, sample)

counter0 = sdmlib.Counter.init_zero(bits, sample)
sdm0 = sdmlib.SDM(address_space, counter0, radius, scanner_type)

counter1 = sdmlib.Counter.init_zero(bits, sample)
sdm1 = sdmlib.SDM(address_space, counter1, radius, scanner_type)

bs_to_boards = {}
boards_to_bs = {}

In [7]:
sdmX = SDMPlayer('X', sdm0, sdm1, bs_to_boards, boards_to_bs)
sdmO = SDMPlayer('O', sdm0, sdm1, bs_to_boards, boards_to_bs)
rndX = Player('X')
rndO = Player('O')
humO = HumanPlayer('O')

In [8]:
def run(pA, pB, n, show=False, debug=False, shuffle=True):
    wins = defaultdict(int)
    for i in range(n):
        winner, seq = play(pA, pB, shuffle=shuffle)

        wins[winner] += 1

        clear_output(wait=True)
        print('Game #{:5d}: {}  {} {}'.format(i+1, list(wins.items()), list(pA.stats.items()), list(pB.stats.items())))

        if debug:
            for i, board in enumerate(seq):
                print('step={}'.format(i))
                print_board(board)
                
        if show:
            print('')
            print_board(seq[-1])
            print('')

        pA.on_finish(winner, seq)
        pB.on_finish(winner, seq)

In [10]:
%time run(sdmX, sdmO, 1000, show=True)

Game # 1000: [('X', 469), (None, 33), ('O', 498)]  [('sdm', 6836), ('random', 778)] [('sdm', 6855), ('random', 787)]

|OO |
|OX |
|XXX|


CPU times: user 11min 49s, sys: 38.6 s, total: 12min 27s
Wall time: 4min 41s


In [14]:
%time run(sdmX, rndO, 1000, show=True)

Game # 1000: [('X', 581), (None, 99), ('O', 320)]  [('sdm', 15207), ('random', 7092)] []

|OOX|
| XO|
|XXO|


CPU times: user 5min 59s, sys: 28.2 s, total: 6min 27s
Wall time: 2min 59s


In [24]:
%time run(rndX, rndO, 100, show=True)

Game #  100: [('X', 35), (None, 15), ('O', 50)]  [] []

|OOX|
|OXX|
|O  |


CPU times: user 319 ms, sys: 86.7 ms, total: 406 ms
Wall time: 483 ms


In [23]:
run(sdmX, humO, 1, show=True, debug=True)

Game #    1: [('O', 1)]  [('sdm', 2176), ('random', 653)] []
step=0
|   |
|   |
|   |

step=1
|   |
| O |
|   |

step=2
|   |
| OX|
|   |

step=3
|   |
| OX|
|  O|

step=4
|   |
| OX|
|X O|

step=5
|O  |
| OX|
|X O|


|O  |
| OX|
|X O|




In [30]:
counter11 = sdmlib.Counter.init_zero(bits, sample)
sdm11 = sdmlib.SDM(address_space, counter11, radius, scanner_type)

In [31]:
playerX = SDMPlayer('O', sdm0, sdm11, bs_to_boards, boards_to_bs)

In [33]:
%time run(sdmX, playerX, 1000, show=True)

Game # 1000: [('X', 390), (None, 98), ('O', 512)]  [('sdm', 21749), ('random', 4710)] [('sdm', 4338), ('random', 1869)]

|OOO|
|X X|
|   |


CPU times: user 14min 3s, sys: 48.2 s, total: 14min 51s
Wall time: 5min 30s
